# Ablation Study with PyTorch
## Titanic example

In [23]:
import inspect
import torch
import torch.nn as nn
import copy
from torch.utils.data import DataLoader


class Ablator:
    def __init__(self, model, dataset, dataloader_kwargs, training_fn):
        # TODO maybe you can have a check that model must be a nn.Sequential, otherwise you make it sequential
        self.model = model
        self.dataset = dataset
        # TODO dataset_features might call it ablation space? or maybe it can be included in Maggy Dataset?
        # Maybe not necessary at all?
        # self.dataset_features = dataset_features
        self.dataloader_kwargs = dataloader_kwargs
        self.training_fn = training_fn

        self.trials = []
        self.state_dictionary = model.state_dict()

    def ablate_layers(self, idx_list, input_shape, infer_activation=False):
        if idx_list is None:
            return copy.deepcopy(self.model)
            # Why a copy? Because if you perform a multiple feature ablation without layer ablation you train on the
            #  same model over and over again.
        if type(idx_list) == int:
            idx_list = [idx_list]
        elif type(idx_list) != list:
            raise TypeError("idx_to_ablate should be an integer or a list of integers")

        new_modules = self._get_module_list()

        if infer_activation:
            activations_idx = []
            for idx in idx_list:
                if ((idx + 1) < len(new_modules)) and self._is_activation(new_modules[idx + 1]):
                    activations_idx.append(idx + 1)
            idx_list = idx_list + activations_idx
            idx_list = list(set(idx_list))

        ablated_modules = self.remove_modules(new_modules, idx_list)
        correct_modules = self._match_model_features(ablated_modules, input_shape)
        ablated_model = nn.Sequential(*correct_modules)

        return ablated_model

    @staticmethod
    def _match_model_features(model_modules, input_shape):
        # TODO you have to do a lot of testing with different pytorch layers
        tensor_shape = (1,) + input_shape
        last_valid_out_features = tensor_shape[1]
        i = 0
        input_tensor = torch.rand(tensor_shape)
        anti_stuck_idx = 0

        while i < len(model_modules):
            layer = model_modules[i]

            try:
                output_tensor = layer(input_tensor)
                anti_stuck_idx = 0
                last_valid_out_features = output_tensor.shape[1]
                # print(layer, "\t\t", output_tensor.shape)
                i += 1
                input_tensor = output_tensor

            except RuntimeError:
                anti_stuck_idx += 1

                if anti_stuck_idx > 1:
                    raise RuntimeError("Ablation failed. Check again what modules you are ablating")

                layer_type = type(layer)
                layer_signature = inspect.signature(layer_type)
                parameters = dir(layer) & layer_signature.parameters.keys()
                new_args = dict()

                for key, value in layer.__dict__.items():
                    if key in parameters:
                        new_args[key] = value

                if "in_features" in new_args:
                    new_args["in_features"] = last_valid_out_features

                elif "in_channels" in new_args:
                    new_args["in_channels"] = last_valid_out_features

                # This new initialization is necessary because even if you change the shape of the layer,
                #  without initialization you don't have the correct number of weights
                model_modules[i] = layer_type(**new_args)
        return model_modules

    def new_trial(self, input_shape, ablated_layers=None, ablated_features=None, infer_activation=False):
        # TODO you don't really need the whole input shape but just the initial features actually
        self.trials.append(Trial(input_shape, ablated_layers, ablated_features, infer_activation))

    def execute_trials(self):
        for i, trial in enumerate(self.trials):
            print("Starting trial", i)

            original_data = self.dataset.data

            # 1) Ablate layers
            ablated_model = self.ablate_layers(trial.ablated_layers, trial.input_shape, trial.infer_activation)

            # 2) Ablate features
            if trial.ablated_features is not None:
                print("Ablating features:", trial.ablated_features)
                self.dataset.ablate_feature(trial.ablated_features)

            # 3) Match features in model
            self._match_model_features(ablated_model, trial.input_shape)

            # 4) Train
            dataloader = DataLoader(self.dataset, **self.dataloader_kwargs)
            trial.metric = self.training_fn(ablated_model, dataloader)
            print("Final metric:", trial.metric, "\n\n")

            # 5) Restore original data
            self.dataset.data = original_data

    def _get_module_list(self):
        modules = []
        for mod in self.model.modules():
            # TODO this is just a quick patch but you should find a better solution
            if not str(mod).startswith("Sequential"):
                modules.append(mod)
        # In PyTorch the first module is actually a description of the whole model
        modules.pop(0)
        return modules

    def remove_modules(self, modules_list, modules_to_ablate):
        for i in reversed(sorted(modules_to_ablate)):
            self._ablate_and_print(modules_list, i)
        return modules_list

    @staticmethod
    def _ablate_and_print(modules, i):
        ablated = modules.pop(i)
        print("Ablating ", i, " - ", ablated, sep="")

    # TODO static methods might be probably refactored to a module utils.py
    @staticmethod
    def _is_activation(layer):
        from torch.nn.modules import activation
        activation_functions = inspect.getmembers(activation, inspect.isclass)
        activation_functions = [x[0] for x in activation_functions]
        if layer.__class__.__name__ in activation_functions:
            return True
        else:
            return False


class MaggyDataset:
    """
    In PyTorch there is no way to get the entire dataset starting from the classes Dataset or DataLoader.
     This is because the only method whose implementation is guaranteed is __getitem__ (enumerate) but there is
     no specification on what this method should return. For instance, it could return a row of a tabular dataset,
     as well as a tuple (label, row). For this reason we necessitate a method that returns a tabular dataset
    (tabular because we define feature ablation only on tabular datasets for now) on which we can ablate the columns.
    """
    def __init__(self, data):
        self.data = data

    def ablate_feature(self, feature):
        raise NotImplementedError


class Trial:
    def __init__(self, input_shape, ablated_layers, ablated_features, infer_activation):
        self.ablated_layers = ablated_layers
        self.ablated_features = ablated_features
        self.input_shape = input_shape
        self.infer_activation = infer_activation
        self.metric = None


In [2]:
import numpy as np
import pandas as pd
import copy
import inspect

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

from torch.autograd import Variable

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

## Data preprocessing

In [3]:
train_path = "../titanic_train.csv"
test_path = "../titanic_test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [6]:
# Join datasets

all_df = pd.concat([train, test])
all_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# For the test dataset "Survived" is NaN

all_df.iloc[50]

PassengerId                            51
Survived                                0
Pclass                                  3
Name           Panula, Master. Juha Niilo
Sex                                  male
Age                                     7
SibSp                                   4
Parch                                   1
Ticket                            3101295
Fare                              39.6875
Cabin                                 NaN
Embarked                                S
Name: 50, dtype: object

In [8]:
# We can see that the last column is type object so we cast it to string

all_df['Embarked'] = all_df['Embarked'].astype(str)
print(all_df.dtypes)

PassengerId      int64
Survived       float64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [9]:
# Drop useless columns

all_df = all_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [10]:
# Additional info for dataset ablation

dataset_features = list(all_df.columns.values)

In [11]:
# Encode labels

cat_cols = ['Pclass', 'Sex', 'SibSp', 'Parch']
for col in cat_cols:
    all_df[col] = LabelEncoder().fit_transform(all_df[col])

In [12]:
# In case you wanted to consider feature Embarked too

#all_df['Embarked'] = LabelEncoder().fit_transform(all_df['Embarked'])

In [13]:
# Replace NaNs

all_df = all_df.fillna(all_df.mean())

In [14]:
# Normalize data

import sklearn.preprocessing

min_max_scaler = sklearn.preprocessing.MinMaxScaler()
all_df = min_max_scaler.fit_transform(all_df)

In [15]:
# Split in train/test again

train_df = all_df[:train.shape[0]]
test_df = all_df[train.shape[0]:]

In PyTorch, there are two important classes for data handling: Dataset and DataLoader  

**Dataset** is the parent class from which your own dataset class inherits. It keeps the data in a numpy format and overrides two methods \_\_getitem\_\_ and \_\_len\_\_ (len is optional) that will be called when you iterate through the dataset.  

**DataLoader** is the class that is used to iterate through the dataset and feed it to your model. You need to specify some parameters during creation like the dataset itself and hyperparameters such as batch_size, shuffle, num_workers, etc.

For this class *TitanicDataset* we add two more requirements that stems from inheriting *MaggyDataset*:
1. All the data should be saved under the attribute `data` of your dataset
2. The dataset class should override the method `ablate_feature` for every possible feature to ablate. This method shouldn't return anything: it should just eliminate a feature from `dataset.data`


In [16]:
dataset_features

['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']

In [17]:
class TitanicDataset(Dataset, MaggyDataset):
    def __init__(self, data):
        super(TitanicDataset).__init__()
        self.data = data

    # Optional override
    def __len__(self):
        return self.data.shape[0]

    # Compulsory override
    def __getitem__(self, idx):
        return self.data[idx]

    def ablate_feature(self, feature):
        # The user defines this function that explains the Ablator how to ablate a specific feature.
        # Not just columns can be ablated but even image channels and anything that is defined in the ablation space.
        global dataset_features
        idx = dataset_features.index(feature)
        self.data = np.delete(self.data, idx, axis=1)

In [18]:
dataloader_kwargs = {"batch_size":64, "shuffle":False}

dataset = TitanicDataset(train_df)

In [19]:
model = nn.Sequential(nn.Linear(6, 64),
                      nn.ReLU(),
                      nn.Linear(64, 64),
                      nn.ReLU(),
                      nn.Linear(64, 32),
                      nn.ReLU(),
                      nn.Linear(32, 32),
                      nn.ReLU(),
                      nn.Linear(32, 2),
                      nn.ReLU(),
                      nn.Linear(2, 1),
                      nn.Sigmoid()
                      )

In [20]:
def training_function(model, dataloader):
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    training_history = []
    epochs = 5

    for epoch in range(epochs):

        epoch_loss = 0
        epoch_accuracy = 0

        for i, data in enumerate(dataloader):
            # print("load number {}".format(i))

            inputs = data[:, 1:]
            labels = data[:, 0]
            inputs, labels = Variable(inputs), Variable(labels)

            inputs = inputs.float()
            labels = labels.float().view(-1, 1)

            optimizer.zero_grad()
            y_pred = model(inputs)

            loss = criterion(y_pred, labels)
            epoch_loss += loss

            accuracy = ((y_pred > 0.5).float() == labels).float().mean()
            epoch_accuracy += accuracy

            loss.backward()
            optimizer.step()
        print("Epoch:{}, Loss:{}, Accuracy:{}".format(epoch,
                                                      epoch_loss.item(),
                                                      epoch_accuracy / dataloader.batch_size))
        training_history.append(epoch_loss.item() / len(train_df))
    print("Training complete\n")
    return loss


In [24]:
ablator = Ablator(model, dataset, dataloader_kwargs, training_function)

ablator.new_trial((6,), None, None)
ablator.new_trial((5,), None, "Sex")
ablator.new_trial((6,), 2, None, infer_activation=True)
ablator.new_trial((5,), 4, "Pclass")
ablator.new_trial((6,), [3, 4, 5, 6], None)

ablator.execute_trials()

Starting trial 0
Epoch:0, Loss:9.455788612365723, Accuracy:0.1348194181919098
Epoch:1, Loss:9.439217567443848, Accuracy:0.1348194181919098
Epoch:2, Loss:9.42446517944336, Accuracy:0.1348194181919098
Epoch:3, Loss:9.411325454711914, Accuracy:0.1348194181919098
Epoch:4, Loss:9.399618148803711, Accuracy:0.1348194181919098
Training complete

Final metric: tensor(0.6634, grad_fn=<BinaryCrossEntropyBackward>) 


Starting trial 1
Ablating features: Sex
Epoch:0, Loss:9.459930419921875, Accuracy:0.1348194181919098
Epoch:1, Loss:9.443719863891602, Accuracy:0.1348194181919098
Epoch:2, Loss:9.429322242736816, Accuracy:0.1348194181919098
Epoch:3, Loss:9.416543960571289, Accuracy:0.1348194181919098
Epoch:4, Loss:9.405226707458496, Accuracy:0.1348194181919098
Training complete

Final metric: tensor(0.6641, grad_fn=<BinaryCrossEntropyBackward>) 


Starting trial 2
Ablating 3 - Linear(in_features=64, out_features=32, bias=True)
Ablating 2 - ReLU()
Epoch:0, Loss:9.388978958129883, Accuracy:0.13481941819